In [ ]:
from RAGLibrary import myWidgets, myRAG, checkConstruct, createSchema, faissConvert, embedding
import os
import json
import torch
import faiss
import pickle
import logging
import numpy as np
from typing import List
from typing import Any, Dict, List
import google.generativeai as genai
from transformers import AutoTokenizer, AutoModel
from google.api_core.exceptions import ResourceExhausted
from sentence_transformers import SentenceTransformer, CrossEncoder

In [ ]:
widgets_list = myWidgets.create_name_form()

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
force_download = True

In [ ]:
""" DEFINE """

data   = widgets_list[0] #HBox 1
keys   = widgets_list[1] #HBox 2
choose = widgets_list[2] #HBox 3

embedd_model = widgets_list[3]
search_egine = widgets_list[4]
rerank_model = widgets_list[5]
respon_model = widgets_list[6]
API_drop     = widgets_list[7]
button_box   = widgets_list[8]

# HBox 1
file_name = data.children[0]
file_type = data.children[1]

# HBox 2
data_key = keys.children[0]
embe_key = keys.children[1]

# HBox 3
switch_model = choose.children[0]
merge_otp    = choose.children[1]
path_end_val = choose.children[1]

# Get value
data_folder   = file_name.value
file_type_val = file_type.value

data_key_val  = data_key.value
embe_key_val  = embe_key.value

API_key_val = API_drop.value
switch      = switch_model.value
merge       = merge_otp.value
path_end    = path_end_val.value

embedding_model = embedd_model.value
searching_egine = search_egine.value
reranking_model = rerank_model.value
responing_model = respon_model.value


# Define
base_path = f"../Data/{data_folder}/{file_type_val}_{data_folder}"

json_file_path = f"{base_path}_Database.json"
schema_ex_path = f"{base_path}_Schema.json"
embedding_path = f"{base_path}_Embeds_{merge}"

torch_path  = f"{embedding_path}.pt"
faiss_path  = f"{embedding_path}.faiss"
mapping_path = f"{embedding_path}_mapping.json"
mapping_data = f"{embedding_path}_map_data.json"

FILE_TYPE    = file_type_val
DATA_KEY     = data_key_val
EMBE_KEY     = embe_key_val
SWITCH       = switch
EMBEDD_MODEL = embedding_model
SEARCH_EGINE = searching_egine
RERANK_MODEL = reranking_model
RESPON_MODEL = responing_model

if FILE_TYPE == "Data":
    MERGE = merge
else: 
    MERGE = "no_Merge"

API_KEY = API_key_val

SEARCH_ENGINE = faiss.IndexFlatIP

print("\n")
print(f"Embedder: {EMBEDD_MODEL}")
print(f"Searcher: {SEARCH_EGINE}")
print(f"Reranker: {RERANK_MODEL}")
print(f"Responer: {RESPON_MODEL}")
print(f"Data Key: {DATA_KEY}")
print(f"Embe Key: {EMBE_KEY}")
print(f"Database: {json_file_path}")
print(f"Torch   : {torch_path}")
print(f"Faiss   : {faiss_path}")
print(f"Mapping : {mapping_path}")
print(f"Map Data: {mapping_data}")
print(f"Schema  : {schema_ex_path}")
print(f"Model   : {SWITCH}")
print(f"Merge   : {MERGE}")
print(f"API Key : {API_KEY}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if (SWITCH == "Auto Model"):
    try:
        tokenizer = AutoTokenizer.from_pretrained(EMBEDD_MODEL, force_download=force_download)
        model = AutoModel.from_pretrained(EMBEDD_MODEL, force_download=force_download)
        model = model.to(device)
        print("Model and tokenizer loaded successfully")
    except Exception as e:
        raise
elif (SWITCH == "Sentence Transformer"):
    try:
        # model = SentenceTransformer(EMBEDD_MODEL).to(device)
        model = SentenceTransformer("../../cached_model")
        print("SentenceTransformer loaded successfully")
    except Exception as e:
        raise

print(f"Using: {device}")

In [ ]:
""" PREPROCESS TEXT """

def preprocess_text(text):
    import re
    if isinstance(text, list):
        return [preprocess_text(t) for t in text]
    if isinstance(text, str):
        text = text.strip()
        text = re.sub(r'[^\w\s\(\)\.\,\;\:\-–]', '', text)
        text = re.sub(r'[ ]{2,}', ' ', text)
        return text
    return text

In [ ]:
""" CREATE EMBEDDING """

def create_embedding(texts, batch_size=32):
    try:
        embeddings = model.encode(texts, batch_size=batch_size, convert_to_tensor=True, device=device)
        return embeddings
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print("VRAM overflow. Switching to CPU.")
            model.to("cpu")
            return model.encode(texts, batch_size=batch_size, convert_to_tensor=True, device="cpu")
        raise e

In [ ]:
"""" SEARCH """

"""
Tìm kiếm văn bản liên quan đến câu hỏi sử dụng FAISS IndexFlatIP.

Args:
    query: Câu hỏi dạng văn bản
    embedd_model: Tên mô hình embedding (EMBEDD_MODEL từ DEFINE)
    search_engine: Loại chỉ mục FAISS (SEARCH_ENGINE từ DEFINE)
    faiss_path: Đường dẫn chỉ mục FAISS (faiss_path từ DEFINE)
    mapping_path: Đường dẫn file ánh xạ (mapping_path từ DEFINE)
    data_path: Đường dẫn file dữ liệu text (mapping_data từ DEFINE)
    data_key: Khóa dữ liệu (DATA_KEY, ví dụ: contents)
    device: Thiết bị PyTorch (device từ DEFINE, ví dụ: cuda hoặc cpu)
    k: Số lượng kết quả trả về

Returns:
    Danh sách các kết quả: {"text": văn bản, "faiss_score": điểm FAISS}
"""

def search_faiss_index(
    query: str,
    embedd_model: str,
    faiss_path: str,
    mapping_path: str,
    data_path: str,
    device: str = "cuda",
    k: int = 10
) -> List[Dict[str, Any]]:

    try:
        # model = SentenceTransformer(embedd_model, device=device)
        query_embedding = model.encode(query, convert_to_tensor=True, device=device).cpu().numpy()
        
        # Chuẩn hóa query embedding cho IndexFlatIP
        query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=-1, keepdims=True)
        
        index = faiss.read_index(faiss_path)
                
        with open(mapping_path, 'r', encoding='utf-8') as f:
            key_to_index = json.load(f)

        with open(data_path, 'r', encoding='utf-8') as f:
            data_mapping = json.load(f)
        
        # Tìm kiếm k kết quả gần nhất
        scores, indices = index.search(query_embedding.reshape(1, -1), k)
        
        # Ánh xạ
        results = []
        index_to_key = {v: k for k, v in key_to_index.items()}
        for idx, score in zip(indices[0], scores[0]):
            if idx not in index_to_key:
                continue
            key = index_to_key[idx]
            text_key = key.replace("Merged_embedding", "Merged_text")
            text = data_mapping.get(text_key, "")
            if not text:
                text = next((v for k, v in data_mapping.items() if k.startswith(key.split("Merged_embedding")[0]) and isinstance(v, (str, list))), "")
            text = text if isinstance(text, str) else " ".join(text) if isinstance(text, list) else ""
            if text:
                results.append({
                    "text": text,
                    "faiss_score": float(score),
                    "key": key
                })
        
        return results
    
    except Exception as e:
        print(f"Error during search: {str(e)}")
        raise


In [ ]:
""" RERANK """

"""
Xếp hạng lại kết quả sử dụng mô hình reranker.

Args:
    query: Câu hỏi dạng văn bản
    results: Danh sách kết quả sơ bộ từ search_faiss_index
    reranker_model: Tên mô hình reranker (RERANK_MODEL từ DEFINE)
    device: Thiết bị PyTorch (cuda hoặc cpu)
    k: Số lượng kết quả trả về sau reranking

Returns:
    Danh sách các kết quả: {"text": văn bản, "rerank_score": điểm reranker, "faiss_score": điểm FAISS}
"""

def rerank_results(
    query: str,
    results: List[Dict[str, Any]],
    reranker_model: str,
    device: str = "cuda",
    k: int = 5
) -> List[Dict[str, Any]]:
    try:
        if not results:
            return []
        
        # Tải mô hình reranker
        reranker = CrossEncoder(reranker_model, device=device)
        
        # Tạo cặp [query, text] để rerank
        pairs = [[query, result["text"]] for result in results]
        
        # Tính điểm rerank
        rerank_scores = reranker.predict(pairs)
        
        # Gắn điểm rerank vào kết quả
        for i, score in enumerate(rerank_scores):
            results[i]["rerank_score"] = float(score)
        
        # Sắp xếp theo rerank_score và lấy top k
        sorted_results = sorted(results, key=lambda x: x["rerank_score"], reverse=True)[:k]
        
        # Định dạng kết quả cuối
        final_results = [
            {
                "text": result["text"],
                "rerank_score": result["rerank_score"],
                "faiss_score": result["faiss_score"]
            }
            for result in sorted_results
        ]
        
        return final_results
    
    except Exception as e:
        print(f"Error during rerank: {str(e)}")
        raise

In [ ]:
""" RESPOND """

"""
Lọc kết quả rerank và sinh câu trả lời tự nhiên bằng Gemini 1.5 Pro.

Args:
    query: Câu hỏi dạng văn bản
    results: Danh sách kết quả từ rerank_results ({'text', 'rerank_score', 'faiss_score', 'key'})
    responser_model: Tên mô hình Gemini (mặc định gemini-1.5-pro)
    device: Thiết bị PyTorch (cuda hoặc cpu, chỉ để tương thích)
    score_threshold: Ngưỡng rerank_score để lọc
    max_results: Số kết quả tối đa để tổng hợp
    gemini_api_key: API key của Google AI Studio

Returns:
    Tuple: (câu trả lời tự nhiên, danh sách kết quả được lọc)
"""

def respond_naturally(
    query: str,
    results: List[Dict[str, Any]],
    responser_model: str = "gemini-2.0-flash-exp",
    score_threshold: float = 0.85,
    max_results: int = 3,
    gemini_api_key: str = None
) -> tuple[str, List[Dict[str, Any]]]:

    try:
        # Lọc kết quả theo ngưỡng rerank_score và độ dài văn bản
        filtered_results = [
            r for r in results
            if r["rerank_score"] > score_threshold and len(r["text"]) > 50
        ][:max_results]
        
        if not filtered_results:
            return "Không tìm thấy thông tin phù hợp với câu hỏi.", []
        
        # Ghép văn bản được lọc thành context
        context = "\n".join([r["text"] for r in filtered_results])
        
        genai.configure(api_key=gemini_api_key)
        
        # Kiểm tra trạng thái mô hình
        model = genai.GenerativeModel(responser_model)
        
        # Tạo prompt cho mô hình
        prompt = (
            f"Câu hỏi: {query}\n"
            f"Thông tin: {context}\n"
            f"Trả lời ngắn gọn và tự nhiên bằng tiếng Việt:"
        )
        
        # Sinh câu trả lời
        response = model.generate_content(
            prompt,
            generation_config={
                "max_output_tokens": 200,
                "temperature": 0.7,
                "top_p": 0.9
            }
        )
        
        # Xử lý response
        if hasattr(response, "candidates") and response.candidates:
            candidate = response.candidates[0]
            if hasattr(candidate, "content") and candidate.content.parts:
                response_text = candidate.content.parts[0].text.strip()
            else:
                raise ValueError("Không tìm thấy nội dung trong candidate của Gemini API.")
        else:
            raise ValueError("Response không có candidates.")

        return response_text, filtered_results
    
    except ResourceExhausted as e:
        error_msg = f"Vượt giới hạn API"
        print(error_msg)
        return

In [ ]:
""" MAIN """

print("<< Enter 'exit', 'quit', 'escape', 'bye' or Press ESC to exit >>")
print("Chatbot: Hello there! I'm here to help you =))")

user_input = "Quy định về đào tạo đại học tại trường Thủ đô Hà Nội"

while True:
    try:
        # user_input = input("You: ")
        user_question = preprocess_text(user_input)
        print(f"You: {user_question}")
        os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
        if user_input.strip().lower() in ["exit", "quit", "escape", "bye", ""]:
            print("Chatbot: Goodbye!")
            break
        
        #Bước 1: Search
        preliminary_results = search_faiss_index(
            query= user_question,
            embedd_model=EMBEDD_MODEL,
            faiss_path=faiss_path,
            mapping_path=mapping_path,
            data_path=mapping_data,
            device=device,
            k=10
        )
        
        # Bước 2: Rerank
        reranked_results = rerank_results(
            query= user_question,
            results=preliminary_results,
            reranker_model=RERANK_MODEL,
            device=device,
            k=5,
        )

        # Bước 3: Generate Response
        response, filtered_results = respond_naturally(
            query= user_question,
            results=reranked_results,
            responser_model=RESPON_MODEL,
            score_threshold=0.85,
            max_results=3,
            gemini_api_key=API_KEY
        )

        print("Câu trả lời:")
        print(response)
        user_input = "exit"

    except KeyboardInterrupt:
        print("\nChatbot: Goodbye!")
        break